In [2]:
import pandas as pd 
import numpy as np
from openai import OpenAI
import os 
import json
import time 

In [10]:
df = pd.read_csv('/Users/yangdajing/Desktop/RA/trainingset.csv')

In [21]:
df['quantum_programming'] = np.where((df['quantum_algorithms'] == 1) & (df['quantum_programming'] == 1), 0, 1)

In [ ]:
df

In [22]:
firts_df = df[df['doc_type'] == 'patent']

In [23]:
firts_df.head()

,doc_id,doc_type,year,title,abstract,methods_of_building_qubits,addressing_obstacles,quantum_computing_model,quantum_algorithms,quantum_programming
2267,10002107,patent,2015,Systems and methods for removing unwanted inte...,"Systems, devices, articles, methods, and techn...",1,0,0,1,0
2268,10002328,patent,2016,Electron spin-based information shuttling for ...,A silicon metal-oxide semiconductor device tra...,1,0,0,0,1
2269,10013657,patent,2017,Periodical modulation of longitudinal coupling...,Method and circuit for reading a value {circum...,1,0,0,0,1
2270,10031791,patent,2018,Systems and methods for quantum computation us...,Systems and methods for executing a quantum co...,0,0,1,1,1
2271,10031887,patent,2015,Systems and methods for improving the performa...,Techniques for improving the performance of a ...,1,1,0,1,0


In [30]:
import os
from openai import OpenAI
import pandas as pd
import numpy as np
from tqdm import tqdm

# 初始化客户端
client = OpenAI(
    api_key='sk-47d6c7ff8baf42cc9ac62d3b2ead9b4f',
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

def classify_quantum_text(title, abstract):
    """使用阿里大模型进行量子文本分类"""
    
    # 构建系统提示词
    system_prompt = """你是一个专业的量子计算领域专家。你的任务是对给定的论文标题，判断它是否属于以下五个类别,同一个论文可能属于多个分类
    1. methods_of_building_qubits 
    2. addressing_obstacles
    3. quantum_computing_model 
    4. quantum_algorithms 
    5. quantum_programming 
    
    请仔细分析标题和摘要的内容，给出且仅给出最合适的分类和置信度，除此之外你不需要给出任何其他信息。
    请只返回类别名称和置信度，格式如下：
    类别: [类别名称]
    置信度: [0-1之间的数字]
    """
    
    # 构建用户提示词
    user_prompt = f"""请对以下论文进行分类：

标题：{title}

"""
    # 调用模型
    completion = client.chat.completions.create(
        model="qwen-plus",
        messages=[
            {'role': 'system', 'content': system_prompt},
            {'role': 'user', 'content': user_prompt}
        ],
        temperature= 0.01
    )
    
    return completion.choices[0].message.content

def process_sample_data(df, sample_size=10):
    """处理样本数据"""
    # 随机选取50个样本
    sample_df = df.head(5)

    results = []
    for idx, row in tqdm(sample_df.iterrows(), total=len(sample_df), desc="处理样本"):
        try:
            classification = classify_quantum_text(row['title'], row['abstract'])
            results.append({
                'doc_id': row['doc_id'],
                'title': row['title'],
                'abstract': row['abstract'],
                'true_labels': {
                    'methods_of_building_qubits': row['methods_of_building_qubits'],
                    'addressing_obstacles': row['addressing_obstacles'],
                    'quantum_computing_model': row['quantum_computing_model'],
                    'quantum_algorithms': row['quantum_algorithms'],
                    'quantum_programming': row['quantum_programming']
                },
                'model_prediction': classification
            })
        except Exception as e:
            print(f"处理文档 {row['doc_id']} 时出错: {e}")
            results.append({
                'doc_id': row['doc_id'],
                'title': row['title'],
                'abstract': row['abstract'],
                'true_labels': {
                    'methods_of_building_qubits': row['methods_of_building_qubits'],
                    'addressing_obstacles': row['addressing_obstacles'],
                    'quantum_computing_model': row['quantum_computing_model'],
                    'quantum_algorithms': row['quantum_algorithms'],
                    'quantum_programming': row['quantum_programming']
                },
                'model_prediction': '处理失败'
            })
    
    return pd.DataFrame(results)

def evaluate_predictions(results_df):
    """评估模型预测结果"""
    # 解析模型预测结果
    def parse_prediction(pred):
        try:
            lines = pred.split('\n')
            category = lines[0].split(': ')[1].strip()
            confidence = float(lines[1].split(': ')[1].strip())
            return category, confidence
        except:
            return None, None
    
    # 计算评估指标
    total = len(results_df)
    correct = 0
    confidence_scores = []
    
    for _, row in results_df.iterrows():
        if row['model_prediction'] == '处理失败':
            continue
            
        pred_category, confidence = parse_prediction(row['model_prediction'])
        if pred_category and confidence:
            confidence_scores.append(confidence)
            # 检查预测是否正确
            if row['true_labels'][pred_category] == 1:
                correct += 1
    
    # 计算准确率
    accuracy = correct / total if total > 0 else 0
    avg_confidence = np.mean(confidence_scores) if confidence_scores else 0
    
    return {
        '总样本数': total,
        '正确预测数': correct,
        '准确率': accuracy,
        '平均置信度': avg_confidence
    }

# 主程序
if __name__ == "__main__":
    # 读取数据
    
    # 处理样本数据
    results_df = process_sample_data(firts_df)
    
    # 保存详细结果
    results_df.to_csv('model_predictions.csv', index=False)
    
    # 评估结果
    evaluation = evaluate_predictions(results_df)
    
    # 打印评估结果
    print("\n评估结果:")
    for metric, value in evaluation.items():
        print(f"{metric}: {value:.4f}")

处理样本:   0%|          | 0/5 [00:00<?, ?it/s]

处理样本: 100%|██████████| 5/5 [00:07<00:00,  1.53s/it]


评估结果:
总样本数: 5.0000
正确预测数: 3.0000
准确率: 0.6000
平均置信度: 0.8200


In [31]:
results_df.head()

,doc_id,title,abstract,true_labels,model_prediction
0,10002107,Systems and methods for removing unwanted inte...,"Systems, devices, articles, methods, and techn...","{'methods_of_building_qubits': 1, 'addressing_...",类别: addressing_obstacles \n置信度: 0.9 \n\n类别: ...
1,10002328,Electron spin-based information shuttling for ...,A silicon metal-oxide semiconductor device tra...,"{'methods_of_building_qubits': 1, 'addressing_...",类别: methods_of_building_qubits \n置信度: 0.8 \n...
2,10013657,Periodical modulation of longitudinal coupling...,Method and circuit for reading a value {circum...,"{'methods_of_building_qubits': 1, 'addressing_...",类别: methods_of_building_qubits \n置信度: 0.8 \n...
3,10031791,Systems and methods for quantum computation us...,Systems and methods for executing a quantum co...,"{'methods_of_building_qubits': 0, 'addressing_...",类别: methods_of_building_qubits \n置信度: 0.8 \n...
4,10031887,Systems and methods for improving the performa...,Techniques for improving the performance of a ...,"{'methods_of_building_qubits': 1, 'addressing_...",类别: addressing_obstacles \n置信度: 0.8\n\n类别: me...


In [34]:
for row in results_df.itertuples(1):
    print(row.title)
    print(row.abstract)

Systems and methods for removing unwanted interactions in quantum devices
Systems, devices, articles, methods, and techniques for advancing quantum computing by removing unwanted interactions in one or more quantum processor. One approach includes creating an updated plurality of programmable parameters based at least in part on a received value for the characteristic magnetic susceptibility of the qubit in the at least one quantum processor, and returning the updated plurality of programmable parameters. Examples programmable parameters include local biases, and coupling values characterizing the problem Hamilton. Also, for example, a quantum processor may be summarized as including a first loop of superconducting material, a first compound Josephson junction interrupting the first loop of superconducting material, a first coupler inductively coupled to the first loop of superconducting material, a second coupler inductively coupled to the first loop of superconducting material, and a

In [32]:
for row in results_df.itertuples():
    print(row.model_prediction)
    print(row.true_labels)

类别: addressing_obstacles  
置信度: 0.9  

类别: methods_of_building_qubits  
置信度: 0.7
{'methods_of_building_qubits': 1, 'addressing_obstacles': 0, 'quantum_computing_model': 0, 'quantum_algorithms': 1, 'quantum_programming': 0}
类别: methods_of_building_qubits  
置信度: 0.8  

类别: addressing_obstacles  
置信度: 0.6  

类别: quantum_computing_model  
置信度: 0.4
{'methods_of_building_qubits': 1, 'addressing_obstacles': 0, 'quantum_computing_model': 0, 'quantum_algorithms': 0, 'quantum_programming': 1}
类别: methods_of_building_qubits  
置信度: 0.8  

类别: addressing_obstacles  
置信度: 0.6
{'methods_of_building_qubits': 1, 'addressing_obstacles': 0, 'quantum_computing_model': 0, 'quantum_algorithms': 0, 'quantum_programming': 1}
类别: methods_of_building_qubits  
置信度: 0.8  

类别: quantum_computing_model  
置信度: 0.7  

类别: addressing_obstacles  
置信度: 0.6
{'methods_of_building_qubits': 0, 'addressing_obstacles': 0, 'quantum_computing_model': 1, 'quantum_algorithms': 1, 'quantum_programming': 1}
类别: addressing_obstacles